## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the acute nephritis dataset loaded below and analyze the model generated for this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
# columns: 
# Temperature of patient { 35C-42C }
# Occurrence of nausea { yes, no }
# Lumbar pain { yes, no }
# Urine pushing (continuous need for urination) { yes, no }
# Micturition pains { yes, no }
# Burning of urethra, itch, swelling of urethra outlet { yes, no }
# decision: Nephritis of renal pelvis origin { yes, no } 

cols = ['temp', 'nausea', 'lumbar_pain', 'urine_pushing', 'micturition_pains', 'burning', 'nephritis']
nephritis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/acute.csv', names=cols)

Recall that we need to check for missing data and create dummy variables from the non-numeric columns. Perform both steps below:

In [0]:
# answer below:

# columns: 
# Temperature of patient { 35C-42C }
# Occurrence of nausea { yes, no }
# Lumbar pain { yes, no }
# Urine pushing (continuous need for urination) { yes, no }
# Micturition pains { yes, no }
# Burning of urethra, itch, swelling of urethra outlet { yes, no }
# decision: Nephritis of renal pelvis origin { yes, no } 

cols = ['temp', 'nausea', 'lumbar_pain', 'urine_pushing', 'micturition_pains', 'burning', 'nephritis']
nephritis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/acute.csv', names=cols)
nephritis.columns

dfdum = pd.DataFrame()
dummies = ['nausea', 'lumbar_pain', 'urine_pushing', 'micturition_pains','burning']
for x in dummies:
  dfdum = pd.concat([dfdum,  pd.get_dummies(nephritis[x], prefix = x, drop_first = True)], axis = 1)

nephritis['nephritis']= np.where(nephritis['nephritis'] == 'yes', 1, 0)
dfn = pd.concat([dfdum, nephritis['nephritis']], axis = 1)
dfn

,nausea_yes,lumbar_pain_yes,urine_pushing_yes,micturition_pains_yes,burning_yes,nephritis
35,0,1,0,0,0,0
35,0,0,1,1,1,0
35,0,1,0,0,0,0
36,0,0,1,1,1,0
36,0,1,0,0,0,0
...,...,...,...,...,...,...
41,0,1,1,0,1,1
41,0,0,0,0,0,0
41,1,1,0,1,0,1
41,0,1,1,0,1,1


Create a train test split with 20% of the data in the test subsample.

In [0]:
# answer below:
from sklearn.model_selection import train_test_split
X=dfn.drop(['nephritis'] , axis = 1)
y=dfn['nephritis']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)



In [0]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
neighbors = KNeighborsClassifier(n_neighbors=5)


neighbors.fit(X_train,y_train)

y_train_predict = neighbors.predict(X_train)
cmtrain = confusion_matrix(y_train, y_train_predict)

y_test_predict = neighbors.predict(X_test)
cmtest = confusion_matrix(y_test, y_test_predict)
print(cmtrain)
print(cmtest)
print(neighbors.score(X_train, y_train))


[[54  0]
 [ 0 42]]
[[16  0]
 [ 0  8]]
1.0


Scale only the independent variables using the minmax scaler.

In [0]:
# answer below:
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

Create a KNN model for our scaled data with k=5 and report the accuracy score.

In [0]:
# answer below:

import seaborn as sns
from sklearn.metrics import confusion_matrix
neighbors = KNeighborsClassifier(n_neighbors=50)


neighbors.fit(X_train,y_train)
print(neighbors.score(X_train, y_train))
print(neighbors.score(X_test, y_test))

      

0.7291666666666666
0.7916666666666666


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [0]:
# answer below:
def _get_weights(dist, weights):
    """Get the weights from an array of distances and a parameter ``weights``
    Parameters
    ----------
    dist : ndarray
        The input distances
    weights : {'uniform', 'distance', 'sqr_distance', or a callable}
        The kind of weighting used
    Returns
    -------
    weights_arr : array of the same shape as ``dist``
        if ``weights == 'uniform'``, then returns None
    """
    if weights in (None, 'uniform'):
        return None
    elif weights == 'sqr_distance':
        # if user attempts to classify a point that was zero distance from one
        # or more training points, those training points are weighted as 1.0
        # and the other points as 0.0
        if dist.dtype is np.dtype(object):
            for point_dist_i, point_dist in enumerate(weights):
                # check if point_dist is iterable
                # (ex: RadiusNeighborClassifier.predict may set an element of
                # dist to 1e-6 to represent an 'outlier')
                if hasattr(point_dist, '__contains__') and 0. in point_dist:
                    dist[point_dist_i] = point_dist == 0.
                else:
                    dist[point_dist_i] = 1. / sqrt(point_dist)
  return weights

# Jays version:
# def my_distance(weights):
#   i = 1.0 for point_dist_i, point_dist in enumerate(weights): 
#     if hasattr(point_dist, 'contains') and 0. in point_dist: 
#       weights[point_dist_i] = point_dist == 0. 
#     else: weights[point_dist_i] = 1.0 / np.sqrt(point_dist) i = i + 1 
    
#   return weights